In [19]:
import numpy
import cv2
import pyaudio
import math
import mediapipe
drawingModule = mediapipe.solutions.drawing_utils
handsModule = mediapipe.solutions.hands

In [20]:
def openCamera():
    vid = cv2.VideoCapture(0)
    if not (vid.isOpened()):
        print("Could not open video device")
        vid.release()
    return vid

In [21]:
def closeCamera(vid):
    if vid == None or not vid.isOpened():
        return
    vid.release()
    cv2.destroyAllWindows()

In [22]:
def generateSound(bitRate, frequency, length):
    if frequency>bitRate:
        bitRate = frequency + 100
    numberOfFrames = int(bitRate*length)
    restFrames = numberOfFrames % bitRate
    waveData = ''
    for x in range(numberOfFrames):
        waveData = waveData + chr(int(math.sin(x/((bitRate/frequency)/math.pi))*127+128))
    for x in range(restFrames):
        waveData = waveData + chr(128)
    
    stream.write(waveData)
    

In [23]:
def detectRectangle(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow("gray", gray)
    _, threshold = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        cv2.drawContours(gray, [approx], 0, (0), 5)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        if len(approx) == 4:
            return True
    return False

In [24]:
def showCamera():
    vid = openCamera()
    while(True):
        ret, frame = vid.read()
        #cv2.namedWindow("Camera", cv2.WINDOW_FREERATIO)
        #cv2.setWindowProperty("Camera", cv2.WND_PROP_AUTOSIZE, cv2.WINDOW_NORMAL)
        if detectRectangle(frame):
            print("yes")
            generateSound(5000, 10000, 0.1)
        else:
            print("No")
        cv2.imshow("Camera", frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    closeCamera(vid)

showCamera()

In [25]:
fingerTips = [
    handsModule.HandLandmark.THUMB_TIP,
    handsModule.HandLandmark.INDEX_FINGER_TIP,
    handsModule.HandLandmark.MIDDLE_FINGER_TIP,
    handsModule.HandLandmark.RING_FINGER_TIP,
    handsModule.HandLandmark.PINKY_TIP
]

In [26]:
capture = openCamera()
frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
once = True
p = pyaudio.PyAudio()
bitRate=5000
stream = p.open(format=p.get_format_from_width(1), channels=2, rate=bitRate, output=True)
with handsModule.Hands(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=2) as hands:
 
    while (True):
 
        ret, frame = capture.read()
 
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        k = 0
        if results.multi_hand_landmarks != None:
            for handLandmarks in results.multi_hand_landmarks:
                for point in fingerTips:
                    normalizedLandmark = handLandmarks.landmark[point]
                    if normalizedLandmark.y < 0.5:
                        generateSound(bitRate, 10000, 0.2)
                        print("gadha")
                    
                # for point in handsModule.HandLandmark:
 
                #     normalizedLandmark = handLandmarks.landmark[point]
                    pixelCoordinatesLandmark = drawingModule._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y, frameWidth, frameHeight)
 
                    cv2.circle(frame, pixelCoordinatesLandmark, 5, (11 * k, 255, 0), -1)
                #     k+=1
                #     if once:
                #         print(["zzzzzzzzzzzzzzz",point, "zzzzzzzzzzzzzz"])
            once = False
 
        cv2.imshow('Test hand', frame)
 
        if cv2.waitKey(1) == 27:
            break
 
closeCamera(capture)
stream.stop_stream()
stream.close()
p.terminate()

In [27]:
generateSound(5000, 10000, 1)

OSError: [Errno -9988] Stream closed